In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
###### check two data set
import os, sys
sys.path.append("/home/n/nguyenpk/CS6220/otdd")
import otdd
from otdd.pytorch.distance import DatasetDistance
#------------------------------------------------------
# pytorch library

import pandas as pd
import math
import random
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.nn.utils.rnn import pad_sequence
import pickle, pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import torch.nn as nn

import numpy as np
import torch.nn.functional as f

# -------------------------------------------
from transformers import BertModel, BertTokenizer,BertTokenizerFast
import gc
#--------------------------------------------------------
seed = 100
if torch.cuda.is_available():
    generator = torch.Generator('cuda').manual_seed(seed)
    device = "cuda:0"
else:
    generator = torch.Generator().manual_seed(seed)
    device = "cpu"

/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/distance.py:23: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm



[KeOps] Warning : There were warnings or errors compiling formula :
<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.

[KeOps] Warning : 
    The location of Cuda header files cuda.h and nvrtc.h could not be detected on your system.
    You must determine their location and then define the environment variable CUDA_PATH,
    either before launching Python or using os.environ before importing keops. For example
    if these files are in /vol/cuda/10.2.89-cudnn7.6.4.38/include you can do :
      import os
      os.environ['CUDA_PATH'] = '/vol/cuda/10.2.89-cudnn7.6.4.38'
    
[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors compiling formula :
/home/n/nguyenpk/.local/lib/python3.8/site-packages/keopscore/binders/nvrtc/nvrtc_jit.cpp:5:10: fatal error: nvrtc.h: No such file or directory
    5 | #include <nvrtc.h>
      |          ^~~~~~~~~
compilation terminated.

OK
[pyKeOps] Compiling nvrtc binder for pytho

ot.gpu not found - coupling computation will be in cpu


# CREATE THE BERT TRANSFROM


In [3]:
# loading bert 
# -------------------------------------------
from transformers import BertModel, BertTokenizer,BertTokenizerFast
import gc
bert_version = 'bert-large-cased'
tokenizer = BertTokenizer.from_pretrained(bert_version)
model = BertModel.from_pretrained(bert_version)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
conf = {}
conf["pickle_raw_MELD"] = "/home/n/nguyenpk/CS6220/project/NLP_DS_distance/data/MELD_raw.pkl"
conf["pickle_raw_IEMOCAP"] = "/home/n/nguyenpk/CS6220/project/NLP_DS_distance/data/IEMOCAP_raw.pkl"
conf["pickle_raw_Daily"] = "/home/n/nguyenpk/CS6220/project/NLP_DS_distance/data/Daily_Dailog_raw.pkl"

In [5]:
ikey = "IEMOCAP"
idata = pd.read_pickle("data/BERT_embed/{}_bert_embd.pickle".format(ikey))

In [7]:
idata.keys()

dict_keys(['embedd_CLS', 'embedd_AVG', 'label', 'train', 'test', 'valid'])

In [5]:
dict_DS = {} ## --> data_name -> {'all', 'train', 'test', 'dev'}
rs = {}
# Read data from file and process
for ikey in ["IEMOCAP", "Daily", "MELD"]:
    print(ikey)
    idata = pd.read_pickle("data/BERT_embed/{}_bert_embd.pickle".format(ikey))
    data = torch.cat([i.to('cpu') for i in idata["embedd_AVG"]])
    label = torch.tensor(np.concatenate(idata["label"]), dtype = int)
    rs["{}_all".format(ikey)] = (data, label)
    # 
    n_train = len(idata["train"])
    n_test = len(idata["test"])
    n_dev = len(idata["valid"])
    # ----
    data_train = torch.cat([i.to('cpu') for i in idata["embedd_AVG"][:n_train]])
    label_train = torch.tensor(np.concatenate(idata["label"][:n_train]), dtype = int)
    rs["{}_train".format(ikey)] = (data_train, label_train)
    # --- 
    data_test = torch.cat([i.to('cpu') for i in idata["embedd_AVG"][n_train:n_train+n_test]])
    label_test = torch.tensor(np.concatenate(idata["label"][n_train:n_train+n_test]), dtype = int)
    rs["{}_test".format(ikey)] = (data_test, label_test)
    #
    data_dev = torch.cat([i.to('cpu') for i in idata["embedd_AVG"][n_train+n_test:]])
    label_dev = torch.tensor(np.concatenate(idata["label"][n_train+n_test:]), dtype=int)
    rs["{}_dev".format(ikey)] = (data_dev, label_dev)
    # del idata
    # gc.collect()
    # torch.cuda.empty_cache()


IEMOCAP
Daily
MELD


In [6]:
class my_DataSet(Dataset):
    """
     This class create my dataset
    """
    def __init__(self, data):
        self.data = data[0]
        self.targets = torch.tensor(data[1], dtype=int)
        self.len = len(self.targets)
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
    def __len__(self):
        return self.len
    def __delitem__(self, key):
        self.data = np.delete(self.data, key, axis=0)
        self.targets = np.delete(self.targets, key, axis=0)

In [7]:
import os
os.environ['CUDA_PATH'] = '/usr/local/cuda-11'
print("Create the dataset")
dict_myDS = {}
for i in rs.keys():
    dict_myDS[i] = my_DataSet(rs[i])
    print(i, len(dict_myDS[i]), dict_myDS[i].data.shape, dict_myDS[i].targets.shape)

print("Begin compute the distance")
mapping_dict = dict(zip(range(len(dict_myDS)), list(dict_myDS.keys())))
m_matrix = np.zeros([len(mapping_dict), len(mapping_dict)])
for i in mapping_dict.keys():
    for j in mapping_dict.keys():
        print("Begin compute distance between:", mapping_dict[i], mapping_dict[j])
        D_i = dict_myDS[mapping_dict[i]]
        D_j = dict_myDS[mapping_dict[j]]
        dist = DatasetDistance(D_i, D_j,
                                   inner_ot_method = 'gaussian_approx',#gaussian_approx
                                   debiased_loss = True,
                                   p = 2, entreg = 1e-1,
                                   min_labelcount=1,
                                   device='cpu') ## error device!!
        
        d = dist.distance(maxsamples = 1000)
        m_matrix[i,j]  = d.item()
        print("Finish compute distance:", m_matrix[i,j])
print("Begin to saving")

distance = pd.DataFrame(m_matrix, columns=mapping_dict.values(), index=mapping_dict.values())
rs_glove_embedding_OTDD = {'df': distance, 'idx': mapping_dict, 'matrix':m_matrix}
pd.to_pickle(rs_glove_embedding_OTDD, "data/result/rs_BERT_AVG_OTDD.pkl")

Create the dataset
IEMOCAP_all 7433 torch.Size([7433, 1024]) torch.Size([7433])
IEMOCAP_train 5146 torch.Size([5146, 1024]) torch.Size([5146])
IEMOCAP_test 1623 torch.Size([1623, 1024]) torch.Size([1623])
IEMOCAP_dev 664 torch.Size([664, 1024]) torch.Size([664])
Daily_all 102979 torch.Size([102980, 1024]) torch.Size([102979])
Daily_train 87170 torch.Size([87171, 1024]) torch.Size([87170])
Daily_test 7740 torch.Size([7740, 1024]) torch.Size([7740])
Daily_dev 8069 torch.Size([8069, 1024]) torch.Size([8069])
MELD_all 13708 torch.Size([13708, 1024]) torch.Size([13708])
MELD_train 9989 torch.Size([9989, 1024]) torch.Size([9989])
MELD_test 2610 torch.Size([2610, 1024]) torch.Size([2610])
MELD_dev 1109 torch.Size([1109, 1024]) torch.Size([1109])
Begin compute the distance
Begin compute distance between: IEMOCAP_all IEMOCAP_all


/tmp/ipykernel_2535239/2001181946.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(data[1], dtype=int)
/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/moments.py:173: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2318.)
  return self.μ, self.Σ - torch.ger(self.μ.T,self.μ)*self.n/(self.n-1)
0it [00:00, ?it/s]/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/sqrtm.py:54: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has change

Finish compute distance: 33.969486236572266
Begin compute distance between: IEMOCAP_all IEMOCAP_train


Finish compute distance: 33.69279479980469
Begin compute distance between: IEMOCAP_all IEMOCAP_test


Finish compute distance: 36.011444091796875
Begin compute distance between: IEMOCAP_all IEMOCAP_dev


...

Finish compute distance: 72.68603515625
Begin compute distance between: MELD_dev MELD_all


Finish compute distance: 49.16990280151367
Begin compute distance between: MELD_dev MELD_train


Finish compute distance: 52.553611755371094
Begin compute distance between: MELD_dev MELD_test


Finish compute distance: 52.57832717895508
Begin compute distance between: MELD_dev MELD_dev


Finish compute distance: 14.363107681274414
Begin to saving


In [9]:
columns = [
    'IEMOCAP_train',
    'IEMOCAP_test',
    'IEMOCAP_dev',
    'IEMOCAP_all',
    'Daily_train',
    'Daily_test',
    'Daily_dev',
    'Daily_all',
    'MELD_train',
    'MELD_test',
    'MELD_dev',
    'MELD_all',
]
distance.loc[columns, columns]

,IEMOCAP_train,IEMOCAP_test,IEMOCAP_dev,IEMOCAP_all,Daily_train,Daily_test,Daily_dev,Daily_all,MELD_train,MELD_test,MELD_dev,MELD_all
IEMOCAP_train,32.277931,40.894196,43.170723,34.578705,54.217453,54.420845,58.740479,53.878090,61.946701,61.858204,63.981796,61.437073
IEMOCAP_test,40.220375,18.538488,43.048248,35.083717,55.913681,55.968994,59.536686,55.004307,62.242676,62.931885,63.578438,61.528900
IEMOCAP_dev,43.122246,43.090126,3.673385,39.162689,56.453671,57.902344,58.500748,56.267296,63.867805,64.774277,65.754547,63.758999
IEMOCAP_all,33.692795,36.011444,39.022572,33.969486,54.167572,54.684887,58.195526,54.747955,61.561352,62.014416,63.264862,62.183739
Daily_train,54.300270,55.466175,56.647053,54.268005,42.534813,44.176659,42.784935,42.011292,65.266411,66.131081,68.201630,65.447067
Daily_test,54.330116,55.625061,56.332382,54.177826,42.523121,37.916553,44.842506,42.045059,65.567093,67.261559,68.077354,66.751656
Daily_dev,57.042412,58.498466,58.349152,57.948410,43.937019,44.172306,37.097050,43.600845,68.264328,70.233658,71.989319,70.141907
Daily_all,54.904907,55.554230,56.422752,53.318398,41.944279,42.006123,44.194302,41.991234,66.515442,66.659630,68.565681,65.783844
MELD_train,61.717556,62.172729,64.034050,61.035934,65.377357,66.718887,69.671066,65.789886,47.122158,50.958061,52.524208,47.738724
MELD_test,62.289677,62.348434,64.929245,62.445602,66.091202,65.997025,70.406258,67.231071,51.521027,34.979897,52.554962,48.662449


## Create DD topics!
- Can move this part to another place later


In [3]:
from preprocessing import preprocessing_raw as pr
from preprocessing import preprocessing_token as pt
#
import sys
from sysconf import  conf
import pandas as pd, numpy as np, pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os

In [4]:
daily_tool = pr.Daily_Dialog_preprocessing()
conv_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_text.txt")
emo_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_emotion.txt")
act_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_act.txt")
topic_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_topic.txt")
data_raw_DD_topic = daily_tool.reading_DD(conv_path, emo_path, act_path, topic_path)
############ --- create again to make sure all of the data is have same label


In [5]:
len(data_raw_DD_topic['actions'])

13118

In [6]:
ll_d = []
ll_l = []
ll_t = []
ll_a = []
for i in  range(len(data_raw_DD_topic['conversation'])):
    data = data_raw_DD_topic['conversation'][i]
    label = data_raw_DD_topic['emotions'][i]
    topic = data_raw_DD_topic['topics'][i]
    action = data_raw_DD_topic['actions'][i]
    if len(data) != len(label): ## cancel this data
        continue
    ll_d.append(data)
    ll_l.append(label)
    ll_t.append(topic)
    ll_a.append(action)
data_raw_DD_topic = {'conversation': ll_d, 'emotions': ll_l, "actions":ll_a, "topics":ll_t}

pd.to_pickle(data_raw_DD_topic, "data/Daily_topic_raw.pkl")

In [7]:
## create the tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pt.flatten_list(data_raw_DD_topic['conversation']))
max_num_tokens = 300
glove_vector = pt.load_pretrained_glove()
##
word_vector_length = len(glove_vector['the'])
word_index = tokenizer.word_index
inv_word_index = {v: k for k, v in word_index.items()}
num_unique_words = len(word_index)
glv_embedding_matrix = np.zeros((num_unique_words + 1, word_vector_length))
for j in range(1, num_unique_words + 1):
    try:
        glv_embedding_matrix[j] = glove_vector[inv_word_index[j]]
    except KeyError:
        glv_embedding_matrix[j] = np.random.randn(word_vector_length) / 200

Loading GloVe model, this can take some time...


In [8]:
data_token = pt.create_data_token(data_raw_DD_topic, tokenizer, max_num_tokens)
pd.to_pickle(data_token, "data/Daily_topic_token.pkl")

In [16]:
len(data_token['data_token'][9000])

11

In [21]:
i

array([[ 614,    9,   55, ...,    0,    0,    0],
       [   2, 3789,    2, ...,    0,    0,    0],
       [  18,    9,   34, ...,    0,    0,    0],
       ...,
       [ 101,  142,   62, ...,    0,    0,    0],
       [ 198,   27,   83, ...,    0,    0,    0],
       [ 104,   25,    1, ...,    0,    0,    0]], dtype=int32)

In [32]:
#-----------
ll = []
ll_e = []
ll_t = []
rs = {}
vocab_size, e_dim = glv_embedding_matrix.shape
embedding = nn.Embedding(vocab_size,e_dim)
embedding.weight = nn.Parameter(torch.from_numpy(glv_embedding_matrix).float())
for idx,i in  enumerate(data_token['data_token']):
    if len(i) >= 9:
        ll.append(embedding(torch.tensor(i, dtype=int)).sum(axis=1))
        ll_e.append(data_token["emotions"][idx])
        ll_t.append(data_token["topics"][idx])
rs["data"] = ll
rs["emotion"] = ll_e
rs["topics"] = ll_t

pd.to_pickle(rs, "data/Glove_embedding_Daily_topic.pkl")

In [33]:
a = pd.read_pickle("data/Glove_embedding_Daily_topic.pkl")

In [34]:
a.keys()

dict_keys(['data', 'emotion', 'topics'])

In [9]:
from sklearn import preprocessing
class my_DataSet(Dataset):
    """
     This class create my dataset
    """
    def __init__(self, data):
        self.data = data[0]
        self.clf = preprocessing.LabelEncoder()
        self.targets = torch.tensor(self.clf.fit_transform(data[1]), dtype=int)
        self.y_raw = data[1] 
        self.len = len(self.targets)
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
    def __len__(self):
        return self.len
    def __delitem__(self, key):
        self.data = np.delete(self.data, key, axis=0)
        self.targets = np.delete(self.targets, key, axis=0)

In [171]:
rs = pd.read_pickle("data/Glove_embedding_Daily_topic.pkl")
topic_DS = {}
for i in sorted(np.unique(rs["topics"])):
    ll_x = []
    ll_y = []
    for j in rs["topics"]:
        data = rs['data'][j]
        label = rs['emotion'][j]
        ll_x.append(data)
        ll_y.append(label)
    X = torch.cat(ll_x)
    y = torch.tensor(np.concatenate(ll_y), dtype=int)
    topic_DS[i] = my_DataSet((X,y))

In [177]:
mapping_dict = dict(zip(range(len(topic_DS)), list(topic_DS.keys())))
m_matrix = np.zeros([len(topic_DS.keys()), len(topic_DS.keys())])
for i in range(len(topic_DS.keys())):
    for j in range(len(topic_DS.keys())):
        print("Begin compute distance between:", i, j)
        D_i = topic_DS[mapping_dict[i]]
        D_j = topic_DS[mapping_dict[j]]
        dist = DatasetDistance(D_i, D_j,
                                   inner_ot_method = 'gaussian_approx',#gaussian_approx
                                   debiased_loss = True,
                                   p = 2, entreg = 1e-1,
                                   min_labelcount=1,
                                   device='cpu') ## error device!!
        
        d = dist.distance(maxsamples = 1000)
        m_matrix[i,j]  = d.item()
        print("Finish compute distance:", m_matrix[i,j])
print("Begin to saving")

pd.to_pickle(m_matrix, "data/result/rs_glove_TOPIC_OTDD.pkl")

Begin compute distance between: 0 0


0it [00:00, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Finish compute distance: 17.450740814208984
Begin to saving


In [23]:
!ls

00_preprocessing.ipynb	data			   preprocessing  sysconf.py
01_OTDD_glove.ipynb	DD_conv_distance_topic.py  __pycache__
base_method.ipynb	my_DD_ds.pickle		   README.md


In [183]:
pd.DataFrame(m_matrix)

,0,1,2,3
0,25.176716,43.170212,53.225464,34.184540
1,73.965477,20.022312,18.220119,23.618389
2,13.759900,44.118320,15.412777,9.708426
3,12.318611,28.510811,86.713394,17.450741


In [11]:
################## --- using distance to predict the topics!!!
rs = pd.read_pickle("data/Glove_embedding_Daily_topic.pkl")
DD_DS = {}
for i in range(len(rs['data'])):
    idata = rs['data'][i]
    ilabel = rs['emotion'][i]
    DD_DS[i] = my_DataSet((idata, ilabel))
pd.to_pickle(DD_DS, "my_DD_ds.pickle")

In [2]:
m_matrix = np.zeros([len(DD_DS.keys()), len(DD_DS.keys())])

for i in range(len(DD_DS.keys())):
    for j in range(len(DD_DS.keys())):
        print("Begin compute distance between:", i, j)
        D_i = DD_DS[i]
        D_j = DD_DS[j]
        dist = DatasetDistance(D_i, D_j,
                                   inner_ot_method = 'gaussian_approx',#gaussian_approx
                                   debiased_loss = True,
                                   p = 2, entreg = 1e-1,
                                   min_labelcount=1,
                                   device='cpu') ## error device!!
        try:
            d = dist.distance()
            m_matrix[i,j]  = d.item()
        except:
            continue
        print("Finish compute distance:", m_matrix[i,j])
print("Begin to saving")

pd.to_pickle(m_matrix, "data/result/rs_glove_DISTANCE_topic_conv.pkl")

NameError: name 'np' is not defined

In [24]:
len(DD_DS)

13117